In [3]:
import cv2
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import json
from mmcv.transforms import Compose
import numpy as np
from mmdet.utils import get_test_pipeline_cfg

def read_json(json_path):
    with open(json_path) as f:
        data = json.load(f)
    return data

def preprocess(test_pipeline, image):
    if isinstance(image, np.ndarray):
        # Calling this method across libraries will result
        # in module unregistered error if not prefixed with mmdet.
        test_pipeline[0].type = 'mmdet.LoadImageFromNDArray'
    test_pipeline = Compose(test_pipeline)
    return test_pipeline(dict(img=image))


## 2. Load a pretrained FP32 model
For this example, we are going to load a pretrained resnet18 model from torchvision. Similarly, you can load any pretrained PyTorch model instead.

In [4]:
# %cd /content/drive/MyDrive/Aimet-torch/mmdetection-3.3.0/
import torch
from mmdet.apis import DetInferencer

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize([640, 640]),  # Resize
])

DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
CONFIG_PATH = '/teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco.py'
WEIGHTS_PATH = '/teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth'

ROOT_DATASET_DIR = '/teamspace/studios/this_studio/COCO'
IMAGES_DIR = os.path.join(ROOT_DATASET_DIR, 'images')
ANNOTATIONS_JSON_PATH = os.path.join(ROOT_DATASET_DIR, 'annotations/instances_val2017.json')
# ANNOTATIONS_JSON_PATH = "/home/shayaan/Desktop/aimet/my_mmdet/temp.json"

model = DetInferencer(model=CONFIG_PATH, weights=WEIGHTS_PATH, device=DEVICE)

DEVICE

[2024-08-12 08:58:04,829] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible
Loads checkpoint by local backend from path: /teamspace/studios/this_studio/mmdetection/rtmdet_tiny_8xb32-300e_coco_20220902_112414-78e30dcc.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: data_preprocessor.mean, data_preprocessor.std

08/12 08:58:06 - mmengine - WARNING - Failed to search registry with scope "mmdet" in the "function" registry tree. As a workaround, the current "function" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmdet" is a correct scope, or whether the registry is initialized.


/usr/local/lib/python3.10/dist-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


device(type='cuda', index=0)

In [5]:
from mmcv.transforms import Compose
test_evaluator = model.cfg.test_evaluator
test_evaluator.type = 'mmdet.evaluation.CocoMetric' 
test_evaluator.dataset_meta = model.model.dataset_meta
test_evaluator.ann_file = ANNOTATIONS_JSON_PATH
test_evaluator = Compose(test_evaluator)

loading annotations into memory...


Done (t=0.60s)
creating index...
index created!


In [6]:
from mmdet.models.utils import samplelist_boxtype2tensor
from mmengine.registry import MODELS

collate_preprocessor = model.preprocess
predict_by_feat = model.model.bbox_head.predict_by_feat
rescale = True

preprocessor = MODELS.build(model.cfg.model.data_preprocessor)
def add_pred_to_datasample(data_samples, results_list):
    for data_sample, pred_instances in zip(data_samples, results_list):
        data_sample.pred_instances = pred_instances
    samplelist_boxtype2tensor(data_samples)
    return data_samples

In [7]:
import random
from typing import Optional
from tqdm import tqdm
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from mmengine.structures import InstanceData

EVAL_DATASET_SIZE = 5000
CALIBRATION_DATASET_SIZE = 2000
BATCH_SIZE = 40

class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, images_dir, annotations_json_path, transform=None):
        self.transform = transform
        self.images_dir = images_dir
        self.annotations_json = read_json(annotations_json_path)


    def __len__(self):
        return len(self.annotations_json['images'])

    def __getitem__(self, idx):
        image_dict = self.annotations_json['images'][idx]
        image_path = os.path.join(self.images_dir, image_dict['file_name'])
        image_id = image_dict['id']


        pre_processed = collate_preprocessor(inputs=[image_path], batch_size=BATCH_SIZE)
        _, data = list(pre_processed)[0]
        data = preprocessor(data, False)['inputs'][0]

        return image_id, image_path, data

evalDataset = CustomImageDataset(images_dir=IMAGES_DIR, annotations_json_path=ANNOTATIONS_JSON_PATH, transform=transform)

In [8]:
def eval_callback(model: torch.nn.Module, num_samples: Optional[int] = None) -> float:    
    data_loader = DataLoader(evalDataset, batch_size=BATCH_SIZE)

    new_preds = []
    for image_id, image_path, _ in tqdm(data_loader):
        pre_processed = collate_preprocessor(inputs=image_path, batch_size=BATCH_SIZE)
        _, data = list(pre_processed)[0]
        data = preprocessor(data, False)

        preds = model.model._forward(data['inputs'].cuda())
        batch_img_metas = [
        data_samples.metainfo for data_samples in data['data_samples']
        ]
        preds = predict_by_feat(*preds, batch_img_metas=batch_img_metas, rescale=True)
        preds = add_pred_to_datasample(data['data_samples'], preds)
        
        for img_id, pred in zip(image_id, preds):
            pred = pred.pred_instances
            new_pred = InstanceData(metainfo={"img_id": int(img_id)})
            new_pred.bboxes = [np.array(p) for p in pred['bboxes'].cpu()]
            new_pred.labels = pred['labels'].cpu()
            new_pred.scores = pred['scores'].cpu()
            new_preds.append(new_pred)
    
    eval_results = test_evaluator(new_preds)
    bbox_map = eval_results['bbox_mAP']
    with open("/teamspace/studios/this_studio/mmdetection/ptq_stats/eval_acc_fp32.json", "w") as f:
        json.dump(eval_results, f, indent=4)
    return bbox_map

In [9]:
from aimet_torch.quantsim import QuantizationSimModel, QuantScheme
from aimet_torch.model_preparer import prepare_model

dummy_input = torch.rand(1, 3, 640, 640).cuda()


prepared_model = prepare_model(model.model)

sim = QuantizationSimModel(model=prepared_model,
                        quant_scheme=QuantScheme.post_training_tf_enhanced,
                        dummy_input=dummy_input,
                        default_output_bw=8,
                        default_param_bw=8,)


2024-08-12 08:58:10,878 - root - INFO - AIMET
2024-08-12 08:58:16,789 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stem.0.bn.module_batch_norm} 
2024-08-12 08:58:16,790 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stem.1.bn.module_batch_norm_1} 
2024-08-12 08:58:16,791 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stem.2.bn.module_batch_norm_2} 
2024-08-12 08:58:16,792 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.0.bn.module_batch_norm_3} 
2024-08-12 08:58:16,793 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.short_conv.bn.module_batch_norm_4} 
2024-08-12 08:58:16,794 - ModelPreparer - INFO - Functional         : Adding new module for node: {backbone.stage1.1.main_conv.bn.module_batch_norm_5} 
2024-08-12 08:58:16,795 - ModelPreparer - INFO - Functional         : Adding new module f

In [ ]:
# print(str(sim))

In [ ]:
print("CALCULATING FP32 ACCURACY NOW")
accuracy = eval_callback(model)

In [ ]:
print(f'- FP32 accuracy: {accuracy}')

## 3. Run AutoQuant
### Create AutoQuant Object

The AutoQuant feature utilizes an unlabeled dataset to achieve quantization. The class **UnlabeledDatasetWrapper** creates an unlabeled Dataset object from a labeled Dataset.

In [10]:
class UnlabeledDatasetWrapper(Dataset):
    def __init__(self, dataset, num_samples):
        self._dataset = dataset
        self.num_samples = num_samples if num_samples < len(self._dataset) else len(self._dataset)
        self.transform = transform

    def __len__(self):
        return self.num_samples 

    def __getitem__(self, index):
        _, _, transformed_images = self._dataset[index]
        return transformed_images
    

In [11]:
from aimet_torch.auto_quant import AutoQuant
from glob import glob

def new_eval_callback(model: torch.nn.Module, num_samples: Optional[int] = None) -> float:
    data_loader = DataLoader(evalDataset, batch_size=BATCH_SIZE)
    new_preds = []
    for image_id, image_path, _ in tqdm(data_loader):
        pre_processed = collate_preprocessor(inputs=image_path, batch_size=BATCH_SIZE)
        _, data = list(pre_processed)[0]
        data = preprocessor(data, False)
        preds = model(data['inputs'].cuda())
        batch_img_metas = [
        data_samples.metainfo for data_samples in data['data_samples']
        ]
        preds = predict_by_feat(*preds, batch_img_metas=batch_img_metas, rescale=True)
        preds = add_pred_to_datasample(data['data_samples'], preds)
        
        for img_id, pred in zip(image_id, preds):
            pred = pred.pred_instances
            new_pred = InstanceData(metainfo={"img_id": int(img_id)})
            new_pred.bboxes = [np.array(p) for p in pred['bboxes'].cpu()]
            new_pred.labels = pred['labels'].cpu()
            new_pred.scores = pred['scores'].cpu()
            new_preds.append(new_pred)

    eval_results = test_evaluator(new_preds)
    num_file = len(glob("/teamspace/studios/this_studio/mmdetection/ptq_stats/eval_acc_quant_*"))
    with open(f"/teamspace/studios/this_studio/mmdetection/ptq_stats/eval_acc_quant_{num_file}.json", "w") as f:
        json.dump(eval_results, f, indent=4)
    bbox_map = eval_results['bbox_mAP']
    return bbox_map


calibration_data_loader = DataLoader(UnlabeledDatasetWrapper(evalDataset, num_samples=CALIBRATION_DATASET_SIZE))
auto_quant = AutoQuant(prepared_model,
                       dummy_input=dummy_input,
                       data_loader=calibration_data_loader,
                       eval_callback=new_eval_callback,
                       encoding_path="/teamspace/studios/this_studio/aimet/Examples/torch/quantization/sim_model_excluded_modules/rtm_det.encodings")

### Run AutoQuant Inference
This step runs AutoQuant inference. AutoQuant inference will run evaluation using the **eval_callback** with the vanilla quantized model without applying PTQ techniques. This will be useful for measuring the baseline evaluation score before running AutoQuant optimization.

In [12]:
# sim, initial_accuracy = auto_quant.run_inference()
model = auto_quant.run_inference()


2024-08-12 08:58:24,239 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_batch_norm} 


2024-08-12 08:58:24,242 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_mul} 


2024-08-12 08:58:24,244 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_batch_norm_1} 


2024-08-12 08:58:24,246 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_mul_1} 


2024-08-12 08:58:24,249 - ModelPreparer - INFO - Functional         : Adding new module for node: {module_batch_norm_2} 


...

2024-08-12 08:58:38,405 - Quant - INFO - No config file provided, defaulting to config file at /usr/local/lib/python3.10/dist-packages/aimet_common/quantsim_config/default_config.json


2024-08-12 08:58:38,436 - Quant - INFO - Unsupported op type Squeeze


2024-08-12 08:58:38,438 - Quant - INFO - Unsupported op type Mean


2024-08-12 08:58:38,453 - Quant - INFO - Selecting DefaultOpInstanceConfigGenerator to compute the specialized config. hw_version:default


- Batchnorm Folding

In [ ]:
print(f"- Quantized A ccuracy (before optimization): {initial_accuracy}")

### Set AdaRound Parameters (optional)
AutoQuant uses a set of predefined default parameters for AdaRound.
These values were determined empirically and work well with the common models.
However, if necessary, you can also use your custom parameters for Adaround.
In this notebook, we will use very small AdaRound parameters for faster execution.

In [ ]:
from aimet_torch.adaround.adaround_weight import AdaroundParameters

adaround_params = AdaroundParameters(calibration_data_loader, num_batches=len(calibration_data_loader), default_num_iterations=2000)
auto_quant.set_adaround_params(adaround_params)

### Run AutoQuant Optimization
This step runs AutoQuant optimization, which returns the best possible quantized model, corresponding evaluation score and the path to the encoding file.
The **allowed_accuracy_drop** parameter indicates the tolerable amount of accuracy drop. AutoQuant applies a series of quantization features until the target accuracy (FP32 accuracy - allowed accuracy drop) is satisfied. When the target accuracy is reached, AutoQuant will return immediately without applying furhter PTQ techniques. Please refer AutoQuant User Guide and API documentation for complete details.

In [ ]:
model, optimized_accuracy, encoding_path = auto_quant.optimize(allowed_accuracy_drop=0.01)
print(f"- Quantized Accuracy (after optimization):  {optimized_accuracy}")

In [ ]:
# model, optimized_accuracy, encoding_path


---
## Summary

Hope this notebook was useful for you to understand how to use AIMET AutoQuant feature.

Few additional resources
- Refer to the AIMET API docs to know more details of the APIs and parameters
- Refer to the other example notebooks to understand how to use AIMET CLE and AdaRound features in a standalone fashion.